In [2]:
import os
import glob
import json
import pickle
import numpy as np
import scipy.spatial
import open3d as o3d
from tqdm.notebook import tqdm
from mmdet3d.utils import register_all_modules
from mmdet3d.apis import inference_detector, init_model

from functools import partial
from pointcloud_filters import (ROR_fn, SOR_fn, 
                                DROR_fn, DSOR_fn, 
                                LIOR_fn, DDIOR_fn)

In [3]:
register_all_modules()

In [ ]:
filter_fns = {
    # "ROR": partial(ROR_fn, radius=0.25, min_neighbour=3), 
    # "SOR": partial(SOR_fn, num_neighbour=3, std_mul=0.01, distance_upper_bound=np.inf), 
    "DROR": partial(DROR_fn, radius_multiplier=3, azimuth_angle_deg=0.4, min_neighbour=3, min_search_radius=0, max_search_radius=10), 
    "DSOR": partial(DSOR_fn, num_neighbour=3, std_mul=0.01, range_mul=0.05, distance_upper_bound=np.inf), 
    "LIOR": partial(LIOR_fn, snow_detection_range=71.2, intensity_th=1, search_radius=0.1, min_neigbour=3), 
    "DDIOR": partial(DDIOR_fn, num_neighbour=3)}

In [4]:
class ObjectDetection3D:
    def __init__(self):
        config_file = "configs/pointpillars/pointpillars_hv_fpn_sbn-all_8xb4-2x_nus-3d.py"
        checkpoint_file = "hv_pointpillars_fpn_sbn-all_4x8_2x_nus-3d_20210826_104936-fca299c1.pth"
        self.model = init_model(config_file, checkpoint_file, device="cuda:0")
    
    def detectObjects(self, pc:np.ndarray)->np.ndarray:
        results, _ = inference_detector(self.model, pc[:,:5].copy())
        scores_3d = results.pred_instances_3d.scores_3d.cpu().numpy()
        bboxes_3d = results.pred_instances_3d.bboxes_3d.tensor.cpu().numpy()
        labels_3d = results.pred_instances_3d.labels_3d.cpu().numpy()
        
        box_as_list_of_string = []
        for score, bbox, lbl in zip(scores_3d, bboxes_3d, labels_3d):
            if (lbl in gt_lbl2name):
                box = bbox[:7].tolist()+[score]
                box[2] = box[2]+0.5*box[5]
                box = ['{:0.6f}'.format(b) for b in box]
                box = [gt_lbl2name[int(lbl)]] + box
                box_as_list_of_string.append(",".join(box))
        return box_as_list_of_string

In [5]:
# def DSOR_fn(cloud, num_neighbour=5, std_mul=0.01, range_mul=0.05):
#     kdtree = scipy.spatial.KDTree(cloud)
#     distances, indices = kdtree.query(np.array(cloud), k=num_neighbour, workers=12)
    
#     mean_distances = distances[:,1:].sum(axis=1)/(num_neighbour-1)
#     sum_ = mean_distances.sum()
#     sq_sum = (mean_distances*mean_distances).sum()
#     mean = sum_/len(mean_distances)
#     variance = (sq_sum - sum_*sum_ / len(mean_distances)) / (len(mean_distances)-1)
#     stddev = np.sqrt(variance)
#     disance_threshold  = mean + std_mul*stddev
    
#     # print(f"mean: {mean}, varience: {variance}, distance_th: {disance_threshold}")

#     pnt_range = np.linalg.norm(cloud, axis=1)
#     dynamic_threshold = disance_threshold * range_mul * pnt_range
#     inlier_bool = mean_distances < dynamic_threshold
#     return inlier_bool

def closestIdx(x):
    angles = np.linspace(-25,15,32)
    idx = np.argmin(np.abs(angles-x))
    return idx

closestIdx_vectorize = np.vectorize(closestIdx)

def addRingInfo(pc):
    # FOV; 40°, vertical FOV (-25° to +15°)
    r = np.linalg.norm(pc[:,:2], axis=1)
    t = np.degrees(np.arctan(pc[:,2]/r))
    ring = closestIdx_vectorize(t)
    return np.hstack((pc, ring[:,None]))

def getBBoxFromGTCuboids(cuboids, nmin=10):
    bbox_as_list_of_string = []
    for bb_dict in cuboids:
        label = bb_dict['label']
        pos = bb_dict['position']
        dim = bb_dict['dimensions']
        yaw = bb_dict['yaw']
        points_count = bb_dict['points_count']
        if (points_count>=nmin) and (np.abs(pos['x'])<=50) and (np.abs(pos['y'])<=50) and (label in cadc_ids2name):
            box = [pos['x'], pos['y'], pos['z'], dim['y'], dim['x'], dim['z'], yaw]
            box = ['{:0.6f}'.format(b) for b in box]
            box = [cadc_ids2name[label]] + box
            bbox_as_list_of_string.append(",".join(box))
    return bbox_as_list_of_string

In [6]:
cadc_ids = ["Car", "Pedestrian", "Truck", "Bus", "Garbage_Containers_on_Wheels", "Traffic_Guidance_Objects", "Bicycle", "Pedestrian_With_Object", "Horse_and_Buggy", "Animals"]
cadc_ids2name = {
    "Car": "vehicle", 
    "Truck": "vehicle", 
    "Bus": "vehicle",
    "Bicycle": "cycle",
    "Pedestrian": "person", 
    "Pedestrian_With_Object": "person" 
}

In [7]:
nus_id2lbl = {'car': 0, 
              'truck': 1, 
              'trailer': 2, 
              'bus': 3,
              'construction_vehicle': 4,
              'bicycle': 5,
              'motorcycle': 6,
              'pedestrian': 7,
              'traffic_cone': 8,
              'barrier': 9}
nus_lbl2id = {v:k for k, v in nus_id2lbl.items()}

gt_lbl2name  = {0: "vehicle", 1: "vehicle", 2: "vehicle", 3: "vehicle", 4: "vehicle", 5: "cycle", 6: "cycle", 7: "person"}

In [8]:
object_det = ObjectDetection3D()

/home/saikrishna/ML_Projetcs/Libraries/3d-object-detector/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


Loads checkpoint by local backend from path: hv_pointpillars_fpn_sbn-all_4x8_2x_nus-3d_20210826_104936-fca299c1.pth


In [ ]:
folders = sorted(glob.glob("/media/saikrishna/T7/Dataset/cadc_devkit/cadcd/*/00*"))
lidar_path = "labeled/lidar_points/data/"
annotation_filename = "3d_ann.json"

for filter_name in tqdm(filter_fns, desc="Filters"):
    filter_fn = filter_fns[filter_name]
    
    save_loc = os.path.join("results/", f"cadc_preds_w_{filter_name}")
    os.makedirs(save_loc, exist_ok=True)
    
    for folder in tqdm(folders, desc="Folders"):
        lidar_scans = sorted(glob.glob(os.path.join(folder, lidar_path, "*.bin")))

        with open(os.path.join(folder,annotation_filename), 'r') as f:
            annotations = json.load(f) 

        for ind, scan in enumerate(tqdm(lidar_scans, desc="Lidar Scans")):
            name = "-".join(folder.split('/')[-2:] + ["{:010d}".format(ind)])+".txt"

            # gtann = annotations[ind]
            # gts = getBBoxFromGTCuboids(gtann['cuboids'])
            # with open(os.path.join("cadc_gts", name), 'w') as f:
            #     f.write("\n".join(gts))    

            pc = addRingInfo(np.fromfile(scan, dtype=np.float32).reshape(-1, 4))
            inlier = filter_fn(pc[:,:4])
            preds = object_det.detectObjects(pc[inlier])
            with open(os.path.join(save_loc, name), 'w') as f:
                f.write("\n".join(preds))

In [19]:
folders = sorted(glob.glob("/media/saikrishna/T7/Dataset/cadc_devkit/cadcd/*/00*"))
lidar_path = "labeled/lidar_points/data/"
annotation_filename = "3d_ann.json"
    
save_loc = os.path.join("results/", f"cadc_preds_intensity")
os.makedirs(save_loc, exist_ok=True)

for folder in tqdm(folders, desc="Folders"):
    lidar_scans = sorted(glob.glob(os.path.join(folder, lidar_path, "*.bin")))

    with open(os.path.join(folder,annotation_filename), 'r') as f:
        annotations = json.load(f) 

    for ind, scan in enumerate(tqdm(lidar_scans, desc="Lidar Scans")):
        name = "-".join(folder.split('/')[-2:] + ["{:010d}".format(ind)])+".txt"

#         gtann = annotations[ind]
#         gts = getBBoxFromGTCuboids(gtann['cuboids'])
#         with open(os.path.join("cadc_gts", name), 'w') as f:
#             f.write("\n".join(gts))    
    
        pc = addRingInfo(np.fromfile(scan, dtype=np.float32).reshape(-1, 4))
        np.random.shuffle(pc[:,3]) #shuffle intensity
        preds = object_det.detectObjects(pc)
        with open(os.path.join(save_loc, name), 'w') as f:
            f.write("\n".join(preds))

Folders:   0%|          | 0/9 [00:00<?, ?it/s]

Lidar Scans:   0%|          | 0/100 [00:00<?, ?it/s]

/home/saikrishna/miniconda3/envs/mmdet/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552843/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Lidar Scans:   0%|          | 0/100 [00:00<?, ?it/s]

Lidar Scans:   0%|          | 0/100 [00:00<?, ?it/s]

...

Lidar Scans:   0%|          | 0/100 [00:00<?, ?it/s]

Lidar Scans:   0%|          | 0/100 [00:00<?, ?it/s]

Lidar Scans:   0%|          | 0/96 [00:00<?, ?it/s]

Lidar Scans:   0%|          | 0/100 [00:00<?, ?it/s]

Lidar Scans:   0%|          | 0/50 [00:00<?, ?it/s]